# 環境構築
## 目的

`C:\lib_ana\lib.ipynb` から、指定した Pythonライブラリの公開API（外部から呼ぶ想定の関数・クラス等）を抽出し、一覧表（CSV/Markdown/Excel など）として `outputs/` に出力する。

## 設計方針（再現性・保守性）

* Notebook は「操作UI」、実装は `src/` に寄せる（ロジック分離）
* 生成物はすべて `outputs/` に集約（差分管理・掃除が簡単）
* 実行ログは `logs/`（失敗時の原因追跡）
* 一時物は `tmp/`（キャッシュや中間ファイル）

## 提案ツリー

```text
C:\lib_ana\
  lib.ipynb
  .vscode\
    settings.json

  src\
    lib_ana\
      __init__.py
      paths.py          # パス定義
      logging_conf.py   # logging(ログ記録)設定
      api_scan.py       # API抽出(探索/inspect等)
      render.py         # 表の整形(CSV/MD/Excel)

  configs\
    settings.json       # 解析対象・出力形式など（任意）

  outputs\
    api_tables\         # API一覧(表)
    reports\            # レポート(まとめ)
    artifacts\          # 追加生成物(図/HTML等)

  data\
    sample\             # サンプル入力(任意)

  logs\
  tmp\
```


In [4]:
# =========================================
# Project bootstrap (Windows / Jupyter)
# 目的: 解析用のディレクトリ構造を自動生成し、
#      Notebookから src/ をimportできるようにする
# =========================================

from __future__ import annotations

import os
import sys
import json
import platform
from pathlib import Path
from datetime import datetime
import logging

# --- 1) ルート固定（lib.ipynbが C:\lib_ana\lib.ipynb 前提）
PROJECT_ROOT = Path(r"C:\lib_ana").resolve()
NOTEBOOK_PATH = PROJECT_ROOT / "lib.ipynb"

# --- 2) ディレクトリ設計（必要なら増やす）
DIRS = {
    "src": PROJECT_ROOT / "src",
    "pkg": PROJECT_ROOT / "src" / "lib_ana",
    "configs": PROJECT_ROOT / "configs",
    "outputs": PROJECT_ROOT / "outputs",
    "api_tables": PROJECT_ROOT / "outputs" / "api_tables",
    "reports": PROJECT_ROOT / "outputs" / "reports",
    "artifacts": PROJECT_ROOT / "outputs" / "artifacts",
    "data": PROJECT_ROOT / "data",
    "sample": PROJECT_ROOT / "data" / "sample",
    "logs": PROJECT_ROOT / "logs",
    "tmp": PROJECT_ROOT / "tmp",
    "vscode": PROJECT_ROOT / ".vscode",
}

for p in DIRS.values():
    p.mkdir(parents=True, exist_ok=True)

# --- 3) Pythonパッケージ雛形を作る（import可能にする）
init_file = DIRS["pkg"] / "__init__.py"
if not init_file.exists():
    init_file.write_text("# lib_ana package\n", encoding="utf-8")

# --- 4) src を import探索パスへ追加（sys.path）
SRC_PATH = str(DIRS["src"])
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

# --- 5) logging(ログ記録) をセットアップ
log_file = DIRS["logs"] / f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file, encoding="utf-8"),
        logging.StreamHandler(sys.stdout),
    ],
)
logger = logging.getLogger("bootstrap")
logger.info("PROJECT_ROOT=%s", PROJECT_ROOT)
logger.info("Python=%s", sys.version.replace("\n", " "))
logger.info("Platform=%s", platform.platform())
logger.info("Log file=%s", log_file)

# --- 6) ディレクトリ構造を表示
def print_tree(root: Path, max_depth: int = 4) -> None:
    root = root.resolve()
    print(f"\n[tree] {root}\n")
    for path in sorted(root.rglob("*")):
        rel = path.relative_to(root)
        depth = len(rel.parts)
        if depth > max_depth:
            continue
        indent = "  " * (depth - 1)
        marker = "📁" if path.is_dir() else "📄"
        print(f"{indent}{marker} {rel}")

print_tree(PROJECT_ROOT, max_depth=4)


2026-02-04 13:59:13,745 | INFO | bootstrap | PROJECT_ROOT=C:\lib_ana
2026-02-04 13:59:13,747 | INFO | bootstrap | Python=3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
2026-02-04 13:59:13,748 | INFO | bootstrap | Platform=Windows-10-10.0.26200-SP0
2026-02-04 13:59:13,749 | INFO | bootstrap | Log file=C:\lib_ana\logs\run_20260204_135913.log

[tree] C:\lib_ana

📁 .vscode
  📄 .vscode\settings.json
  📄 .vscode\settings.json.bak_20260204_135731
📁 configs
📁 data
  📁 data\sample
📄 lib.ipynb
📁 logs
  📄 logs\run_20260204_135711.log
  📄 logs\run_20260204_135913.log
📁 outputs
  📁 outputs\api_tables
  📁 outputs\artifacts
  📁 outputs\reports
📁 src
  📁 src\lib_ana
    📄 src\lib_ana\__init__.py
📁 tmp


## 目的

* `src/` を Python の解析パスに入れて補完・型解析を効かせる
* Notebook の作業ディレクトリをワークスペースに固定して、相対パス事故を防ぐ
* 保存時の自動整形（環境に整形ツールが入っていれば効く）

## 反映先

* `C:\lib_ana\.vscode\settings.json`



In [5]:
from pathlib import Path
import json
from datetime import datetime

PROJECT_ROOT = Path(r"C:\lib_ana").resolve()
vscode_dir = PROJECT_ROOT / ".vscode"
vscode_dir.mkdir(parents=True, exist_ok=True)
settings_path = vscode_dir / "settings.json"

# 既存があればバックアップ
if settings_path.exists():
    backup = settings_path.with_suffix(f".json.bak_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
    backup.write_text(settings_path.read_text(encoding="utf-8"), encoding="utf-8")

# 推奨設定（環境により拡張機能が無くても害は少ない設定に寄せる）
settings = {
    "python.analysis.extraPaths": ["./src"],
    "jupyter.notebookFileRoot": "${workspaceFolder}",
    "files.eol": "\n",
    "editor.formatOnSave": True,
    # black(コード整形ツール)等がある場合に効く。無くても致命傷にはならない。
    "python.formatting.provider": "black",
}

settings_path.write_text(json.dumps(settings, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"Updated: {settings_path}")
print(settings_path.read_text(encoding='utf-8'))


Updated: C:\lib_ana\.vscode\settings.json
{
  "python.analysis.extraPaths": [
    "./src"
  ],
  "jupyter.notebookFileRoot": "${workspaceFolder}",
  "files.eol": "\n",
  "editor.formatOnSave": true,
  "python.formatting.provider": "black"
}


# ディレクトリ構造確認

In [9]:
import subprocess
import sys

# 1. shell=True: コマンドプロンプト経由で実行
# 2. capture_output=True: 出力を変数に格納
# 3. text=True: テキストとして扱う
# 4. encoding='cp932': Windowsの日本語環境(Shift-JIS)に合わせる
cp = subprocess.run(['tree', '/f'], shell=True, capture_output=True, text=True, encoding='cp932')

if cp.returncode != 0:
    print('tree failed.', file=sys.stderr)
    print(cp.stderr) # エラー内容を表示
else:
    # 成功した場合、キャプチャした標準出力を表示
    print(cp.stdout)

フォルダー パスの一覧
ボリューム シリアル番号は D4CC-DB5B です
C:.
│  lib.ipynb
│  
├─.history
│  │  organize_files_20260204162147.py
│  │  organize_files_20260204162150.py
│  │  
│  └─src
│          analyzer_v2_20260204145440.py
│          analyzer_v2_20260204145442.py
│          analyzer_v2_20260204145533.py
│          analyzer_v2_20260204145547.py
│          analyzer_v2_20260204145549.py
│          analyzer_v2_20260204145550.py
│          analyzer_v2_20260204145704.py
│          analyzer_v2_20260204150317.py
│          analyzer_v3_20260204151629.py
│          analyzer_v3_20260204151633.py
│          library_explorer_20260204144502.py
│          library_explorer_20260204144505.py
│          ui_v2_20260204145455.py
│          ui_v2_20260204145458.py
│          ui_v2_20260204145724.py
│          ui_v2_20260204150328.py
│          ui_v2_20260204150701.py
│          ui_v3_20260204151656.py
│          ui_v3_20260204151700.py
│          
├─.vscode
│      settings.json
│      settings.json.bak_20260204_135731
│   

In [2]:
import sys
import os

# srcディレクトリをパスに追加してインポートできるようにする
src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# 作成したモジュールをインポclearート
# ※ファイル名を変更した場合はここも変更してください
from library_explorer import LibraryExplorerApp

# アプリケーションの起動
app = LibraryExplorerApp()
app.display()

ModuleNotFoundError: No module named 'library_explorer'

In [ ]:
# ファイルパス: C:\lib_ana\lib.ipynb

import sys
import os

# プロジェクトルートのsrcフォルダをパスに追加
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# 開発用リロード処理
import importlib
import ui_v2
import analyzer_v2
importlib.reload(analyzer_v2)
importlib.reload(ui_v2)

# アプリケーション起動
from ui_v2 import CognitiveLibraryUI

app = CognitiveLibraryUI()
app.display()

ModuleNotFoundError: No module named 'ui_v2'

In [ ]:
import sys
import os

# srcパスの追加
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# リロードと起動
import importlib
import ui_v2
importlib.reload(ui_v2)

from ui_v2 import CognitiveLibraryUI
app = CognitiveLibraryUI()
app.display()

ModuleNotFoundError: No module named 'ui_v2'

In [ ]:
from ui_v3 import CognitiveLibraryUIV3
CognitiveLibraryUIV3().display()


ModuleNotFoundError: No module named 'models'

In [ ]:
# ファイルパス: C:\lib_ana\lib.ipynb

import sys
import os

# パス設定
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# リロード（開発用）
import importlib
import ui_v3
import analyzer_v3
importlib.reload(analyzer_v3)
importlib.reload(ui_v3)

from ui_v3 import CognitiveLibraryUI_v3

# アプリ起動
app = CognitiveLibraryUI_v3()
app.display()

ModuleNotFoundError: No module named 'models'

In [ ]:
import sys
sys.path.insert(0, r"C:\lib_ana\src\v2")

In [ ]:
import sys
sys.path.insert(0, r"C:\lib_ana\src\library_explorer_v4")

from ui_v4 import CognitiveLibraryUIV4
CognitiveLibraryUIV4().display()


In [1]:
import os
import sys

# 1) 作業ディレクトリの src を先頭に追加
SRC_DIR = os.path.join(os.getcwd(), "src")

# いったん古い CLE_lib_ana 側のパスを sys.path から除外しておく
sys.path = [p for p in sys.path if "CLE_lib_ana" not in p]

if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# 2) ここで v5 をインポートすると、C:\lib_ana\src\v5 が優先される
from v5.ui_v5 import CognitiveLibraryUIV5

ui = CognitiveLibraryUIV5()
ui.display()


In [ ]:
import os, sys
SRC_DIR = os.path.join(os.getcwd(), "src")
sys.path = [p for p in sys.path if "CLE_lib_ana" not in p]
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

from v5.ui_v5 import CognitiveLibraryUIV5

ui = CognitiveLibraryUIV5()
ui.display()


In [3]:
# -*- coding: utf-8 -*-
# Sample code generated by Cognitive Library Explorer V5
# Target: timesfm.ForecastConfig

from timesfm.configs import ForecastConfig

if __name__ == '__main__':
    result = ForecastConfig(max_context=0, max_horizon=0, normalize_inputs=False, window_size=0, per_core_batch_size=1, use_continuous_quantile_head=False, force_flip_invariance=True, infer_is_positive=True, fix_quantile_crossing=False, return_backcast=False)
    print('Result:', result)

Result: ForecastConfig(max_context=0, max_horizon=0, normalize_inputs=False, window_size=0, per_core_batch_size=1, use_continuous_quantile_head=False, force_flip_invariance=True, infer_is_positive=True, fix_quantile_crossing=False, return_backcast=False)
